In [63]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from os import listdir
import sys
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
#================= config here =================
# database website: https://relational.fit.cvut.cz/dataset/Financial
data_path = "../test_data/financial/"
sys.path.append(data_path)
master_table_name = "loan.csv_account_id"
#================= 
table_names = listdir(data_path)
tables = {} #key:table name; value: table in Pandas.Dataframe
for each in table_names:
    tables[each] = pd.read_csv(data_path+each)

groud_truth_foreignKey = set([("loan.csv_account_id", "account.csv_account_id")
                  , ("order.csv_account_id", "account.csv_account_id")
                  , ("trans.csv_account_id", "account.csv_account_id")
                  , ("disp.csv_account_id", "account.csv_account_id")
                  , ("card.csv_disp_id", "disp.csv_disp_id")
                  , ("disp.csv_client_id", "clinet.csv_client_id")
                  , ("account.csv_district_id", "district.csv_district_id")
                  , ("clinet.csv_district_id", "district.csv_district_id")
                  ])

In [145]:
#======global vars============
# groud_truth_foreignKey
# tables
visited = set() # set of table names, store the tables that already in the queue (to be processed)
delimiter = ".csv_"

def get_names(tableCol):
    """
    Input:
        String: eg. `loan.csv_account_id`
    Output:
        tuple of String: eg. (`loan.csv`, `account_id`)
    """
    split = re.split(delimiter, tableCol)
    table_name = split[0] + delimiter[:-1]
    column_name = split[1]
    return table_name, column_name


def forward(curt_table):
    """
    Input:
        curt_table: String, name of table. eg. `loan.csv_account_id`
    Output:
        result: Pandas.DataFrame, big featurized table (join)
    """
    table_name, key_column_name = get_names(curt_table)
    k_tables = get_forward_tables(table_name)
    result = tables[table_name]
    
    for table in k_tables:
        foreign_table_name = re.split(delimiter, table)[0] + delimiter[:-1]
        table = backward(table)
        print(key_column_name) # name of primary-foreign key 
        result = result.join(table.set_index(key_column_name), on=key_column_name, lsuffix="_"+table_name, rsuffix="_"+foreign_table_name)

    return result


def backward(curt_table):
    """
    Input:
        curt_table: String, name of table. eg. `account.csv_account_id`
    Output:
        result: Pandas.DataFrame, big featurized table (join of groupby count)
    """
    table_name, column_name = get_names(curt_table)
    k_tables = get_backward_tables(table_name)
    result = tables[table_name]
    
    for table in k_tables:
        # aggregated result of : groupby + count()
        table_name, column_name = get_names(table)
        table = tables[table_name]
        r = table.groupby(column_name).count()
        r = r.rename(columns = lambda x : table_name+"_"+x)
        
        result = result.join(r, on=column_name)
        
    return result
        
def get_forward_tables(table_name):
    """
    Output:
        list of String, String is the second element in `groud_truth_foreignKey` tuples (Primary key)
    """
    result = list()
    for relation in groud_truth_foreignKey:
        if (table_name in relation[0]):
            result.append(relation[1])
            visited.add(relation[0])
    return result

def get_backward_tables(table_name):
    """
    Output:
        list of String, String is the first element in `groud_truth_foreignKey` tuples (Primary key)
    """
    result = list()
#     print (visited)
    for relation in groud_truth_foreignKey:
        if (table_name in relation[1] and (relation[0] not in visited)):
            result.append(relation[0])
#             visited.add(relation[0])
    return result

In [148]:
big_table = forward(master_table_name)
big_table.to_csv("featureized_finacial.csv", index=False)

account_id


In [149]:
big_table.keys()

Index([u'loan_id', u'account_id', u'date_loan.csv', u'amount', u'duration',
       u'payments', u'status', u'district_id', u'frequency',
       u'date_account.csv', u'order.csv_order_id', u'order.csv_bank_to',
       u'order.csv_account_to', u'order.csv_amount', u'order.csv_k_symbol',
       u'disp.csv_disp_id', u'disp.csv_client_id', u'disp.csv_type',
       u'trans.csv_trans_id', u'trans.csv_date', u'trans.csv_type',
       u'trans.csv_operation', u'trans.csv_amount', u'trans.csv_balance',
       u'trans.csv_k_symbol', u'trans.csv_bank', u'trans.csv_account'],
      dtype='object')